# Processing guidelies: 
1. Downsample to 512
2. reference to Cz
3. filter ([.1,120] and line noise)
4. remove bad channels (not done here)
5. Epoch data 
6. inspect epochs (not done here)
7. save as mne .fif and as array 


In [1]:
from ast import literal_eval
import gc
import json
import matplotlib
#matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import mne
import numpy as np
import os
import pandas as pd
import pyxdf
import seaborn as sns

In [2]:
datasrc = "/net/store/nbp/projects/GTI_decoding/data/pilot"
alltrials_file = "allTrialsData.csv"
experimentdetails_file = "ExperimentDetails.csv"
montagefile = "/net/store/nbp/projects/GTI_decoding/saurabh/eegDecoding/misc/ANT_EEG_channel_mapping.json"

processed_data_folder = os.path.join(datasrc,'processed')
if not os.path.isdir(processed_data_folder):
    os.mkdir(processed_data_folder)
    

In [3]:
trial_types = ["/".join([x,y,z]) for x in ['lift','use'] for y in ['fam','unfam'] for z in ['left','right']]
trial_event_map = {t:i+1 for i,t in enumerate(trial_types)}

In [4]:
trial_event_map

{'lift/fam/left': 1,
 'lift/fam/right': 2,
 'lift/unfam/left': 3,
 'lift/unfam/right': 4,
 'use/fam/left': 5,
 'use/fam/right': 6,
 'use/unfam/left': 7,
 'use/unfam/right': 8}

In [5]:
expt_df = pd.read_csv(os.path.join(datasrc,experimentdetails_file))
trials_df = pd.read_csv(os.path.join(datasrc,alltrials_file))

with open(montagefile) as f: #open the montage file provided by Debbie
    montage = json.loads(f.read())
#change channel names to match standard naming convention in mne standard montage
montage['ch-1']='Fp1'
montage['ch-2']='Fpz'
montage['ch-3']='Fp2'

Fetch the data and basic summarization

1. Load the block
2. Gather the summary of the EEG channels
3. Gather summary of the triggers
4. Create the event array in mne format
5. Load EEG into mne

In [6]:
def get_channel_names(temp,n):
    ch_names = []
    for i in range(n):
        n = temp[i]['label'][0].replace("ExG","ch-")
        ch_names.append(montage[n] if "ch-" in n else n)
    return ch_names
    
    

In [7]:
def get_eeg_and_preprocess(df):
    out = 'ok'
    event_streams = literal_eval(df['trig_streams'].item())
    eeg_stream = df['eeg_stream'].item()
    stim_stream = df['stim_stream'].item()
    if (eeg_stream == 99) |  (stim_stream == 99) | ('99' in event_streams):
        print(" All datastreams not found. Skipped".format(df['subj_id'],df['block']))
        out = 'skipped'
    else:
        try:
            print("start...")
            data, header = pyxdf.load_xdf(df['filename'].item())
            eeg_time = data[eeg_stream]["time_stamps"] - data[eeg_stream]["time_stamps"][0]
            sr = data[eeg_stream]['info']['effective_srate']
            n_ch = int(data[eeg_stream]['info']['channel_count'][0])
            ch_typ= ['eeg'] * n_ch
            ch_names = get_channel_names(
                data[eeg_stream]['info']['desc'][0]['channels'][0]['channel'],
                n_ch)
            
            #map the trial start cue i.e. the first index in the event_streams to ecent array for mne
            tdf = trials_df.loc[(trials_df['subj_id']==df['subj_id'].item()) & 
                                (trials_df['block']==df['block'].item()),
                               'condition']
            tdf = tdf.map(trial_event_map)
            x = np.searchsorted(data[eeg_stream]["time_stamps"],
                                data[int(event_streams[0])]["time_stamps"])
            events = np.zeros((x.shape[0],3),dtype=int)
            events[:,0] = x
            events[:,2] = tdf.to_numpy(dtype=int)
            
            #create mne info
            info = mne.create_info(ch_names, ch_types=ch_typ, sfreq=sr)
            
            #mne data array should be in (nChannel,nSamples) whereas xdf stores in (nSamples,nChannel)
            eeg = mne.io.RawArray(np.transpose(data[eeg_stream]["time_series"]), info) 
            #drop auxiliaries and not needed channels
            eeg.drop_channels(['BIP65', 'BIP66', 'BIP67', 'BIP68', 'AUX69', 'AUX70', 'AUX71', 'AUX72'])
            #set the montage
            eeg.set_montage('standard_1020')
            #resample to 512Hz
            #Note: the raw and the event are resampled simultaneously so that they stay more or less in synch.
            
            eeg_resamp, events_resamp = eeg.resample(sfreq=512,events=events)
            
            del eeg
            gc.collect()
            
            #EEG is recorded with average reference. Re-refer to Cz
            eeg_resamp.set_eeg_reference(ref_channels=['Cz'])
            #Filter resampled EEG
            #High pass 0.1 Hz
            #Low pass 120 Hz
            #notch 50Hz,100Hz
            eeg_resamp.filter(l_freq=0.1, h_freq=120)
            eeg_resamp.notch_filter(freqs=[50,100])
            
            #epochs paramaters
            tmin = -0.1 # 100 msec before the event boundary
            tmax = 6.0 # each trial is 2.0 + 0.5 + 3.0 + 0.5...NOTE: first 0.5 sec of action is included
            baseline = (tmin,0) #i.e. the entire zone from tmin to 0
            epochs = mne.Epochs(eeg_resamp, events_resamp, event_id=trial_event_map, tmin=tmin, tmax=tmax)
            epochs.drop_bad()
            
        except Exception as e:
            print(e)
            out = 'failed'
    
    if out == 'ok':
        print("Success")
        return epochs, out
    else:
        print(out)
        return None, out

    

In [12]:
subjects = expt_df['subj_id'].unique()
results = []
for s in subjects:
    subj_df = expt_df[expt_df['subj_id']==s]
    for block in subj_df['block'].unique():
        print("\nProcessing Subject {} Block {} ##############".format(s,block))
        epoch_return, flag = get_eeg_and_preprocess(subj_df[subj_df['block']==block])
        if flag == 'ok':
            filename = s + '_' + str(block) + '_epo'
            epoch_return.save(os.path.join(processed_data_folder,filename + '.fif'), overwrite=True)
            #convert to numpy array
            epoch_array = epoch_return.get_data()
            with open(os.path.join(processed_data_folder,filename + '.npy'), 'wb') as f:
                np.save(f, epoch_array)
            del epoch_return, epoch_array
            gc.collect()
        results.append("Subject: " + s + " Block: " + str(block) + " Report: " + flag)
        print("...end")
    
print("Finished")         


Processing Subject P001 Block 1 ##############
start...
1023.9999941251851
9
Creating RawArray with float64 data, n_channels=72, n_times=741888
    Range : 0 ... 741887 =      0.000 ...   724.499 secs
Ready.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 16897 samples (33.002 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter

Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 3124 original time points ...
0 bad epochs dropped
Success
Loading data for 1 events and 3124 original time points ...
Loading data for 48 events and 3124 original time points ...
Loading data for 48 events and 3124 original time points ...
...end

Processing Subject P002 Block 3 ##############
start...
1023.9999952107912
9
Creating RawArray with float64 data, n_channels=72, n_times=734208
    Range : 0 ... 734207 =      0.000 ...   716.999 secs
Ready.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower 

- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 3124 original time points ...
0 bad epochs dropped
Success
Loading data for 1 events and 3124 original time points ...
Loading data for 48 events and 3124 original time points ...
Loading data for 48 events and 3124 original time points ...
...end

Processing Subject P003 Block 2 ##############
start...
1024.0000079606132
1
Creating RawArray with float64 data, n_channels=72, n_times=783904
    Range : 0 ... 783903 =      0.000 ...   765.530 secs
Ready.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) referenc

- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 16897 samples (33.002 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Not setting metadata
Not setting metadata
48 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 48 events and 3124 original time points ...
0 bad epochs dropped
Success
Loading data for 1 events and 3124 original time points ...
Loading data for 48 events and 3124 original time points .

Ready.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 16897 samples (33.002 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- 

In [13]:
for r in results:
    print(r)

Subject: P001 Block: 1 Report: ok
Subject: P001 Block: 2 Report: ok
Subject: P001 Block: 3 Report: ok
Subject: P002 Block: 1 Report: ok
Subject: P002 Block: 2 Report: ok
Subject: P002 Block: 3 Report: ok
Subject: P002 Block: 4 Report: ok
Subject: P002 Block: 5 Report: ok
Subject: P002 Block: 6 Report: ok
Subject: P003 Block: 1 Report: ok
Subject: P003 Block: 2 Report: ok
Subject: P003 Block: 3 Report: ok
Subject: P003 Block: 4 Report: ok
Subject: P003 Block: 5 Report: ok
Subject: P003 Block: 6 Report: ok
Subject: P004 Block: 1 Report: skipped
Subject: P004 Block: 2 Report: ok
Subject: P004 Block: 3 Report: ok
Subject: P004 Block: 4 Report: ok
Subject: P004 Block: 5 Report: ok
Subject: P004 Block: 6 Report: ok
